<a href="https://colab.research.google.com/github/suna0107/ANN_DL101/blob/main/%E2%98%85(250325)_1_HS10_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**[HS CODE Representation]**

- AutoEncoder 기반 Feature Extraction
- 100개차원 → 64개차원 → 32개 차원 → 64개차원 → 100개차원
- 32개 차원의 값을 추출

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import os
import pandas as pd

# 1. 데이터 준비

##1. HS Code Embedding Layer (10자리 × 10차원 = 100차원)



##1.(def) 엑셀에서 HS Code 불러오기 및 데이터 전처리

In [ ]:
# ✅ 엑셀 파일 경로
file_path = "/content/HS_CODE_PROCESSED_kr_en_final.xlsx"

# ✅ 엑셀 로드
df = pd.read_excel(file_path, dtype=str)

# ✅ 'HS_CODE' 열을 리스트로 저장
# 컬럼명을 제외한 실제 데이터만 가져오기
hs_codes = df["HS_CODE"].iloc[1:].tolist()  # 또는 df["HS_CODE"].dropna().tolist()

##2. (def정의) HS Code를 정수 리스트 -> PyTorch 텐서로 변환


7283940512" → [7,2,8,3,9,4,0,5,1,2]

In [ ]:
def preprocess_hs_code(hs_codes):
    processed_codes = []
    for hs in hs_codes:
        digits = [int(digit) for digit in hs]
        processed_codes.append(digits)
    return torch.tensor(processed_codes, dtype=torch.long) # PyTorch 텐서로 변환

In [ ]:
# 함수 실행
hs_tensor = preprocess_hs_code(hs_codes)
hs_tensor.shape

torch.Size([16820, 10])

#2. Autoencoder 모델

- HSAutoencoder 클래스 설정 : HS Code를 Autoencoder를 이용해 학습하고, 32차원의 Latent Representation을 추출하는 모델



In [ ]:
torch.manual_seed(42)  # 재현성을 위해 랜덤시드 설정
class HSAutoencoder(nn.Module):
    def __init__(self, vocab_size=10, embed_dim=10, input_dim=100, hidden_dim=64, latent_dim=32):
        super(HSAutoencoder, self).__init__() # __init__(생성자) = 네트워크 구조 정의
        self.embedding = nn.Embedding(vocab_size, embed_dim) # 1. HS Code Embedding Layer (10자리 × 10차원 = 100차원)
        self.encoder = nn.Sequential(       # 2️. Encoder: 100 → 64 → 32
            nn.Linear(input_dim, hidden_dim),# 100 → 64
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim), # 64 → 32
            nn.ReLU()
        )
        self.decoder = nn.Sequential(      # 3. Decoder: 32 → 64 → 100/ 잠재 공간(32차원)을 다시 100차원으로 복원
            nn.Linear(latent_dim, hidden_dim),# 32 → 64
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),# 64 → 100
            nn.Sigmoid() # Sigmoid()를 사용 → 출력값을 0~1 범위로 정규화
        )

    def forward(self, x): # 4. forward() → 순전파(입력 → 인코딩 → 디코딩)
        x = self.embedding(x) # HS Code를 벡터화
        x = x.view(x.size(0), -1) # 100차원으로 펼침 (batch_size, 100)
        encoded = self.encoder(x) # 인코딩 (100 → 32)
        decoded = self.decoder(encoded) # 디코딩 (32 → 100)
        return decoded, encoded # 디코딩된 결과(100차원) 와 32차원 잠재 표현을 반환

# 3.DataLoader 설정
 - 배치단위로 데이터 분할하여 train data 준비

In [ ]:
# 1. 데이터준비
file_path = "/content/HS_CODE_PROCESSED_kr_en_final.xlsx"  # 엑셀 파일 경로
hs_tensor = preprocess_hs_code(hs_codes)
hs_codes = df["HS_CODE"].tolist()

#2. 배치단위로 데이터 준비 : dataset
#2-1. hs_tensor를 PyTorch에서 사용할 수 있도록 변환 = 모델 입력 데이터
# hs_tensor는 원본 데이터(텐서), dataset은 PyTorch 모델 학습을 위한 포맷
batch_size = 32
dataset = TensorDataset(hs_tensor)

#2. 배치 단위로 데이터 분할
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
## batch_size=32 : 한 번에 32개씩 모델에 입력
## shuffle=True: 데이터 순서를 랜덤
# train_loader : dataset에서 데이터를 32개씩 랜덤하게 구성하여 학습



#4. 모델 학습(Training)

In [ ]:
#1. 모델 초기화
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HSAutoencoder().to(device) # 모델을 device에 할당 = 모델을 GPU 또는 CPU에 배치
criterion = nn.MSELoss() #손실 함수(MSE)
optimizer = optim.Adam(model.parameters(), lr=0.001) # Adam Optimizer 설정

hs_tensor = hs_tensor.to(device) #  hs_tensor 입력 데이터를 GPU/CPU로 이동

In [ ]:
# 학습루프 설정
num_epochs = 100
for epoch in range(num_epochs): # 100번(num_epochs=100) 반복하며 모델을 학습
    optimizer.zero_grad() # 기존의 그래디언트 초기화
    reconstructed, latent = model(hs_tensor) # 모델 순전파 수행

# `hs_tensor`를 모델 출력(reconstructed)과 동일한 100차원으로 변환
    hs_tensor_embedded = model.embedding(hs_tensor)  #(batch_size, 10, 10)
    hs_tensor_embedded = hs_tensor_embedded.view(hs_tensor_embedded.size(0), -1)  #(batch_size, 100)

# MSE 손실 계산
    loss = criterion(reconstructed, hs_tensor_embedded.float())
    loss.backward() # 역전파 수행
    optimizer.step() # 가중치 업데이트
    if epoch % 10 == 0: # 10 에포크마다 손실 값 출력
        print(f"Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [0/100], Loss: 1.4987
Epoch [10/100], Loss: 1.4188
Epoch [20/100], Loss: 1.1885
Epoch [30/100], Loss: 0.9826
Epoch [40/100], Loss: 0.9459
Epoch [50/100], Loss: 0.9263
Epoch [60/100], Loss: 0.9076
Epoch [70/100], Loss: 0.8878
Epoch [80/100], Loss: 0.8653
Epoch [90/100], Loss: 0.8394




-  현재 손실값이 0.8 정도로 수렴

- 일반적으로 0.1 ~ 1.0 사이면 괜찮은 복원 성능일 가능성이 높음


## 4-1. 학습 손실 줄이기 위한 방안

(1) 학습률 조정 : lr=0.001 → 0.0005 -> 0.00025
손실 : 0.8 -> 0.66 -> 0.56

- 최적의 학습률로 loss 적정한 값 설정 중요
- 최종 학습률 : 0.0005 로 설정



In [ ]:
# 학습률 조정 : 0.0005로 감소시킴
optimizer = optim.Adam(model.parameters(), lr=0.0005)
# 학습률 조정 : 0.00025로 감소시킴
# optimizer = optim.Adam(model.parameters(), lr=0.00025)

In [ ]:
# 학습루프 설정
num_epochs = 100
for epoch in range(num_epochs): # 100번(num_epochs=100) 반복하며 모델을 학습
    optimizer.zero_grad() # 기존의 그래디언트 초기화
    reconstructed, latent = model(hs_tensor) # 모델 순전파 수행

# `hs_tensor`를 모델 출력(reconstructed)과 동일한 100차원으로 변환
    hs_tensor_embedded = model.embedding(hs_tensor)  #(batch_size, 10, 10)
    hs_tensor_embedded = hs_tensor_embedded.view(hs_tensor_embedded.size(0), -1)  #(batch_size, 100)

# MSE 손실 계산
    loss = criterion(reconstructed, hs_tensor_embedded.float())
    loss.backward() # 역전파 수행
    optimizer.step() # 가중치 업데이트
    if epoch % 10 == 0: # 10 에포크마다 손실 값 출력
        print(f"Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [0/100], Loss: 0.8106
Epoch [10/100], Loss: 0.7948
Epoch [20/100], Loss: 0.7804
Epoch [30/100], Loss: 0.7669
Epoch [40/100], Loss: 0.7539
Epoch [50/100], Loss: 0.7414
Epoch [60/100], Loss: 0.7291
Epoch [70/100], Loss: 0.7162
Epoch [80/100], Loss: 0.7032
Epoch [90/100], Loss: 0.6906


##5. HS Code의 32차원 Representation

In [ ]:
# HS Code의 32차원 Representation 확인 : latent_vectors
_, latent_vectors = model(hs_tensor) # model(hs_tensor) Autoencoder의 forward() 함수가 실행
# 출력값은 (복원된 데이터, 32차원 Representation)
## 언더스코어(_)는 복원된 데이터(Decoder 출력)를 말고, 오직 latent_vectors만 저장

latent_vectors.shape

torch.Size([16820, 32])

In [ ]:
print("\n🔹 HS Code Embedding Representation (32차원):")
# PyTorch Tensor → NumPy 배열 변환 출력 가능하게 만듦
latent_vectors_np = latent_vectors.detach().cpu().numpy()
latent_vectors_np


🔹 HS Code Embedding Representation (32차원):


array([[ 3.9277961,  0.       , 14.768636 , ...,  1.226641 ,  9.8383045,
        13.410532 ],
       [ 4.7087317,  0.       , 11.197586 , ...,  0.9700618,  6.2565813,
        11.431807 ],
       [ 5.4289045,  0.       ,  7.855844 , ...,  1.9067473,  5.978879 ,
        10.671864 ],
       ...,
       [ 2.666396 ,  0.       , 10.368039 , ...,  1.8679845,  8.0265465,
         6.0774355],
       [ 2.66522  ,  0.       , 12.301055 , ...,  2.238477 ,  8.579593 ,
         8.481747 ],
       [ 2.6851647,  0.       , 11.8448925, ...,  1.9224209,  8.362972 ,
         8.700762 ]], dtype=float32)

- PyTorch 파일 (.pt) → PyTorch 모델에서 사용할 때 편리

In [ ]:
torch.save(latent_vectors, "hs_latent_vectors.pt")


- CSV 파일 (to_csv) → 다른 데이터와 연계할 때 가장 쉬움

In [ ]:
# Pandas DataFrame으로 변환
df_latent = pd.DataFrame(latent_vectors_np)
df_latent.shape
# CSV 파일로 저장
df_latent.to_csv("hs_latent_vectors.csv", index=False)

- hs_code열 추가 : 다른 데이터와 merge하기 위한 데이터

In [4]:
# 1. 오토인코더 결과 벡터 로딩
ae_vecs = pd.read_csv("/content/hs_latent_vectors.csv")  # 각 행 = 32차원 latent 벡터

# 2. 원래 학습에 사용한 HS 코드 리스트
hs_code_list = pd.read_excel("/content/HS_CODE_PROCESSED_kr_en_final.xlsx")



In [6]:
hs_code_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16820 entries, 0 to 16819
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   4                             16820 non-null  int64  
 1   6                             16820 non-null  int64  
 2   8                             16820 non-null  int64  
 3   10                            16820 non-null  int64  
 4   HS_CODE                       16820 non-null  int64  
 5   제XX류                          16820 non-null  object 
 6   제XXXX.XX호                     16710 non-null  object 
 7   Chapter                       16820 non-null  int64  
 8   Subheading                    16710 non-null  float64
 9   국문                            16820 non-null  object 
 10  영문                            16820 non-null  object 
 11  Referenced_HS                 16820 non-null  object 
 12  Referenced_HS_Description_EN  378 non-null    object 
 13  R

In [7]:
# 3. 벡터에 HS 코드 정보 붙이기
ae_vecs['HS_CODE_10'] = hs_code_list['HS_CODE']
ae_vecs['HS_CODE_6'] = ae_vecs['HS_CODE_10'].astype(str).str[:6]

# 4. 결과 저장
ae_vecs.to_csv("hs_ae_vectors_with_code.csv", index=False)